In [1]:
import telebot
from telebot import types
import os

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

import urllib.request

from keras.models import Model, Sequential, model_from_json
from keras.layers import Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from PIL import Image

from keras.preprocessing.image import load_img, save_img, img_to_array
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing import image
import matplotlib.pyplot as plt

### Data.

In [46]:
data = pd.read_csv('academy_awards.csv', index_col=[0])
data['gender'].replace({'male': 1, 'female': 0}, inplace=True)
data = data.sort_values('itemLabel')
data.head(2)

,item,itemLabel,award,img url,gender,date of birth,place of birth,citizenship
50,http://www.wikidata.org/entity/Q104514,Adrien Brody,best actor,https://upload.wikimedia.org/wikipedia/commons...,1,Sat Apr 14 00:00:00 1973,Woodhaven,United States of America
22,http://www.wikidata.org/entity/Q41163,Al Pacino,best actor,https://upload.wikimedia.org/wikipedia/commons...,1,Thu Apr 25 00:00:00 1940,Manhattan,United States of America


### Model.

In [16]:
"""
Build the VGG Face model
"""

model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(Convolution2D(4096, (7, 7), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(4096, (1, 1), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(2622, (1, 1)))
model.add(Flatten())
model.add(Activation('softmax'))

In [24]:
"""
‘Standing on the shoulder of giants’ - transfere pre-trained weights
"""
model.load_weights('vgg_face_weights.h5')

In [25]:
"""
Preprocess photo
"""
def preprocess_img(image_path):
    img = load_img(image_path, target_size=(224, 224))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img

"""
FINDING SIMILARITIES between vectors

Cosine & Euclidean distances
"""
def CosSimilarity(source_representation, test_representation):
    a = np.matmul(np.transpose(source_representation), test_representation)
    b = np.sum(np.multiply(source_representation, source_representation))
    c = np.sum(np.multiply(test_representation, test_representation))
    return 1 - (a / (np.sqrt(b) * np.sqrt(c)))

def EuclDistance(source_representation, test_representation):
    euclidean_distance = source_representation - test_representation
    euclidean_distance = np.sum(np.multiply(euclidean_distance, euclidean_distance))
    euclidean_distance = np.sqrt(euclidean_distance)
    return euclidean_distance

In [26]:
"""
Representing images as 2622 dimensional vector
"""

vgg_face = Model(inputs=model.layers[0].input, outputs=model.layers[-2].output)

In [82]:
"""
Set the threshold value for this face recognition model
"""

epsilon = 0.42

def verifyFace(img1, img2, user_gender, actor_gender):

    img1_rep = np.concatenate([vgg_face.predict(preprocess_img(img1))[0,:], np.array([user_gender])])
    img2_rep = np.concatenate([vgg_face.predict(preprocess_img('photos/%s' % (img2)))[0,:], np.array([user_gender])])
    
    cos_similarity = CosSimilarity(img1_rep, img2_rep)
    eucl_distance = EuclDistance(img1_rep, img2_rep)
    return cos_similarity, eucl_distance

### Bot.

In [2]:
with open('bot_token.txt') as f:
    token = f.read()

In [35]:
def gender_age():
    with open('gender.txt') as f:
        gender = int(f.read())
    with open('age.txt') as f:
        age = int(f.read())
    return gender, age

In [81]:
bot = telebot.TeleBot(token)

def get_gender(messageid):
    keyboard = types.InlineKeyboardMarkup()
    # По очереди готовим текст и обработчик для каждого знака зодиака 
    key_M = types.InlineKeyboardButton(text='Male(M)', callback_data='M') 
    # И добавляем кнопку на экран 
    keyboard.add(key_M)
    key_F = types.InlineKeyboardButton(text='Famale(Ж)', callback_data='F')
    keyboard.add(key_F)
    bot.send_message(messageid, text='Укажи свой пол', reply_markup=keyboard)
    return

@bot.message_handler(commands=['start', 'help'])
def start(message):

    get_gender(message.from_user.id)
    
@bot.message_handler(content_types=['text'])
def get_age(message):
    with open('age.txt', 'w') as f:
        f.write(message.text)
    print(message.text)
    bot.send_message(message.from_user.id, text='Теперь залей свое фото')
    

@bot.message_handler(content_types=['photo'])
def get_photo(message):
    
    raw = message.photo[0].file_id
    path = raw+".jpg"
    file_info = bot.get_file(raw)
    downloaded_file = bot.download_file(file_info.file_path)
    with open('photo.jpg','wb') as new_file:
        new_file.write(downloaded_file)
    bot.send_message(message.from_user.id, text='Твоя награда...')
    bot.send_message(message.from_user.id, text='Да уж, с таким лицом в призы сложно попасть, но надо подождать...')
    user_gender, user_age = gender_age()
    print(user_gender, type(user_gender))
    
    simdf_lst = []
    
    user_img_path = 'photo.jpg'
    for i, actor_img_path in enumerate(os.listdir('photos/')[:50]):
        
        actor_gender = data['gender'].iloc[i]
        cos_sim, euclid = verifyFace(user_img_path, actor_img_path, user_gender, actor_gender)
        temp_df = pd.DataFrame({'cos_sim': [cos_sim],
                                'euclid': [euclid],
                                'photo_index': [i]
                               })
        simdf_lst.append(temp_df)
    simdf = pd.concat(simdf_lst)
    simdf.to_csv('simdf.csv')
    best_cos = simdf['cos_sim'].argmax()
    award = data['award'].iloc[best_cos].capitalize()
    bestsimactor = os.listdir('photos/')[best_cos].split('.')[0]
    bot.send_message(message.from_user.id, text=f'{award}!!!')
    bot.send_message(message.from_user.id, text=f'Как {bestsimactor}!')
    actor_photo = open('photos/'+os.listdir('photos/')[best_cos], 'rb')
    bot.send_photo(message.from_user.id, actor_photo)
    
    
@bot.callback_query_handler(func=lambda call: True)
def callback_worker(call): # Если нажали на одну из 2 кнопок
    if call.data == 'M':
        with open('gender.txt', 'w') as f:
            f.write('1')
    if call.data == 'F':
        with open('gender.txt', 'w') as f:
            f.write('0')
    bot.send_message(call.from_user.id, text='Укажи возраст')
                      
bot.polling(none_stop=False, interval=0, timeout=20)



34
1 <class 'int'>
[1]
(2623,)
[1]
(2623,)
[1]
(2623,)
[1]
(2623,)
[1]
(2623,)
[1]
(2623,)
[1]
(2623,)
[1]
(2623,)
[1]
(2623,)
[1]
(2623,)
[1]
(2623,)
[1]
(2623,)
[1]
(2623,)
[1]
(2623,)
[1]
(2623,)
[1]
(2623,)
[1]
(2623,)
[1]
(2623,)
[1]
(2623,)
[1]
(2623,)
[1]
(2623,)
[1]
(2623,)
[1]
(2623,)
[1]
(2623,)
[1]
(2623,)
[1]
(2623,)
[1]
(2623,)
[1]
(2623,)
[1]
(2623,)
[1]
(2623,)
[1]
(2623,)
[1]
(2623,)
[1]
(2623,)
[1]
(2623,)
[1]
(2623,)
[1]
(2623,)
[1]
(2623,)
[1]
(2623,)
[1]
(2623,)
[1]
(2623,)
[1]
(2623,)
[1]
(2623,)
[1]
(2623,)
[1]
(2623,)
[1]
(2623,)
[1]
(2623,)
[1]
(2623,)
[1]
(2623,)
[1]
(2623,)
[1]
(2623,)
